In [1]:
import numpy as np
import pandas as pd
import tracktor as tr
import matplotlib.pyplot as plt
import cv2
import sys
from sklearn.cluster import KMeans
from scipy.optimize import linear_sum_assignment
from scipy.spatial.distance import cdist

In [2]:
df = pd.read_csv("/Users/lukefields/Desktop/Shrimp-Capstone/tracktor-master/output/zebrafish_video_tracked.csv")
df.head()

,Unnamed: 0,frame,pos_x,pos_y,id
0,0,3.0,783.514170,62.967611,A
1,1,3.0,174.192848,60.541507,B
2,2,3.0,303.360000,119.509630,C
3,3,3.0,193.814035,293.143860,D
4,4,3.0,226.002179,333.840959,E


### Now, we want to generate a dataframe that has all 25 different combos (the distance between 1 and 1, the distance between 1 and 2, ...., the distance between 5 and 5) 
### So, we want a dataframe that has two columns (one being the combo index, the other being the actual combo (i.e. AA) 
### Then, we will iterate through that

In [3]:
# This outputs all of the unique ids in our dataset
# In other words, if we ware working with 4 shrimp, we would get ids 1, 2, 3, 4
# If we are working with 5 shrimp labeled by letters, we would get ids A, B, C, D, E

diff_ids = df["id"].unique()
print(diff_ids)

['A' 'B' 'C' 'D' 'E']


##### We will use the itertools package in order to create the different combos between shrimp

In [4]:
# Products
# unique combos (yields 25)
from itertools import product
for combo in product(diff_ids, repeat = 2):  # 2 for pairs, 3 for triplets, etc
    print(combo)
    
    
    
# Permutations
# unique combos (yields 20)
from itertools import permutations
for combo in permutations(diff_ids, 2):  # 2 for pairs, 3 for triplets, etc
    print(combo)

    
    
# Combinations with replacement
# unique combos (yields 15)
from itertools import combinations_with_replacement
for combo in combinations_with_replacement(diff_ids, 2):  # 2 for pairs, 3 for triplets, etc
    print(combo)
    

    
# Combinations with no replacement
# unique combos (Yields 10 for 5 shrimp), we will want this
from itertools import combinations
for combo in combinations(diff_ids, 2):  # 2 for pairs, 3 for triplets, etc
    print(combo)

('A', 'A')
('A', 'B')
('A', 'C')
('A', 'D')
('A', 'E')
('B', 'A')
('B', 'B')
('B', 'C')
('B', 'D')
('B', 'E')
('C', 'A')
('C', 'B')
('C', 'C')
('C', 'D')
('C', 'E')
('D', 'A')
('D', 'B')
('D', 'C')
('D', 'D')
('D', 'E')
('E', 'A')
('E', 'B')
('E', 'C')
('E', 'D')
('E', 'E')
('A', 'B')
('A', 'C')
('A', 'D')
('A', 'E')
('B', 'A')
('B', 'C')
('B', 'D')
('B', 'E')
('C', 'A')
('C', 'B')
('C', 'D')
('C', 'E')
('D', 'A')
('D', 'B')
('D', 'C')
('D', 'E')
('E', 'A')
('E', 'B')
('E', 'C')
('E', 'D')
('A', 'A')
('A', 'B')
('A', 'C')
('A', 'D')
('A', 'E')
('B', 'B')
('B', 'C')
('B', 'D')
('B', 'E')
('C', 'C')
('C', 'D')
('C', 'E')
('D', 'D')
('D', 'E')
('E', 'E')
('A', 'B')
('A', 'C')
('A', 'D')
('A', 'E')
('B', 'C')
('B', 'D')
('B', 'E')
('C', 'D')
('C', 'E')
('D', 'E')


In [5]:
# This gets an array of all of the different combos

zebra_combo = []
for combo in combinations(diff_ids, 2):  # 2 for pairs, 3 for triplets, etc
    zebra_combo.append(combo)
    
print(zebra_combo)
len(zebra_combo)

[('A', 'B'), ('A', 'C'), ('A', 'D'), ('A', 'E'), ('B', 'C'), ('B', 'D'), ('B', 'E'), ('C', 'D'), ('C', 'E'), ('D', 'E')]


10

In [6]:
type(("A", "B"))

tuple

In [7]:
ex = ("A", "B")
first = ex[0]
second = ex[1]
second

'B'

In [9]:
zebra_combo[0][0]
zebra_combo[0][1]

'B'

#### Let's try and build this function, eh?

In [27]:
# This for loop assigns shrimp ids by iterating through the tuple
for pair in zebra_combo:
    id1 = pair[0]
    id2 = pair[1]

The first id is A and the second id is B
The first id is A and the second id is C
The first id is A and the second id is D
The first id is A and the second id is E
The first id is B and the second id is C
The first id is B and the second id is D
The first id is B and the second id is E
The first id is C and the second id is D
The first id is C and the second id is E
The first id is D and the second id is E


#### Let's start with a function that takes in a dataframe and creates all of the unique combos

In [28]:
def master_funct_combo(dataframe):
    diff_ids = df["id"].unique()
    shrimp_combo = []
    for combo in combinations(diff_ids, 2):  # 2 for pairs, 3 for triplets, etc
        shrimp_combo.append(combo)
        
    return shrimp_combo
        
master_funct_combo(df)

[('A', 'B'),
 ('A', 'C'),
 ('A', 'D'),
 ('A', 'E'),
 ('B', 'C'),
 ('B', 'D'),
 ('B', 'E'),
 ('C', 'D'),
 ('C', 'E'),
 ('D', 'E')]

#### Ok, mission accomplished there, now let's start assembling our function in its entirety

In [35]:
from itertools import combinations
# output_filepath = '/Users/lukefields/Desktop/Shrimp-Capstone/tracktor-master/videos/' + video + '_tracked.csv'
# Output filepath will be something like the template below
# "/Users/yourname/Desktop/shripfolder/where-ever-it-is-you-want-your-output/" + shrimp1 + shrimp2 + "_distances.csv"
# Or a realized version, Luke Fields's specific file path for where he wants the output
# "/Users/lukefields/Desktop/Shrimp-Capstone/tracktor-master/output/zebrafish_video_tracked.csv"

def master_funct_combo(dataframe, output_filepath):
    diff_ids = dataframe["id"].unique()
    shrimp_combo = []
    for combo in combinations(diff_ids, 2):  # 2 for pairs, 3 for triplets, etc
        shrimp_combo.append(combo)
    
    for pair in shrimp_combo:
        shrimpX = pair[0]
        shrimpY = pair[1]
        
        # This will subset our dataframe into one with just the two animals we care about
        dataframe = dataframe[(dataframe["id"] == shrimpX) | (dataframe["id"] == shrimpY)]

        for idx, ID in enumerate(np.unique(dataframe['id'])):
            dataframe['id'][dataframe['id'] == ID] = idx

        distances = []
        for fr in np.unique(dataframe['frame']):
            tmp = dataframe[dataframe['frame'] == fr]
            x = tmp[tmp['id'] == 0]['pos_x'].values[0] - tmp[tmp['id'] == 1]['pos_x'].values[0]
            y = tmp[tmp['id'] == 0]['pos_y'].values[0] - tmp[tmp['id'] == 1]['pos_y'].values[0]
            distances.append(np.sqrt(x**2 + y**2))

        timestamp = np.unique(dataframe['frame'])/fps
        
        pair_list = []
        for i in range(len(distances)):
            pair_list.append(pair)

        dist_df = pd.DataFrame([pair_list, timestamp, distances]).transpose()
        dist_df.columns = ["Pair", "Timestamp", "Distance"]
        
        new_output_filepath = output_filepath + str(shrimpX) + str(shrimpY) + "_distances.csv"

        dist_df.to_csv(output_filepath, sep=',')

#### Ok, and now let's try and do some work on those individual functions

In [ ]:
def dist_between_combo(dataframe, fps): 
    id1 = zebra_combo[0]
    id2 = zebra_combo[0]
    
    # This will subset our dataframe into one with just the two animals we care about
    dataframe = dataframe[(dataframe["id"] == id1) | (dataframe["id"] == id2)]
    
    for idx, ID in enumerate(np.unique(dataframe['id'])):
        dataframe['id'][dataframe['id'] == ID] = idx
        
    distances = []
    for fr in np.unique(dataframe['frame']):
        tmp = dataframe[dataframe['frame'] == fr]
        x = tmp[tmp['id'] == 0]['pos_x'].values[0] - tmp[tmp['id'] == 1]['pos_x'].values[0]
        y = tmp[tmp['id'] == 0]['pos_y'].values[0] - tmp[tmp['id'] == 1]['pos_y'].values[0]
        distances.append(np.sqrt(x**2 + y**2))
        
    timestamp = np.unique(dataframe['frame'])/fps
    
    dist_df = pd.DataFrame([timestamp, distances]).transpose()
    dist_df.columns = ["Timestamp", "Distance"]
    
    plt.scatter(np.unique(dataframe['frame'])/fps, distances, c='#32CD32', s=5, alpha=0.5)
    plt.xlabel('Time (sec)', fontsize=16)
    plt.ylabel('Distance from Each Other', fontsize=16)
    plt.tight_layout()
    plt.savefig('imgs/ex3_fig2.eps', format='eps', dpi=300)
    plt.show()
    print(dist_df.head())
    
    return dist_df

In [ ]:
def master_funct(dataframe, fps):
    for pair in shrimp_combo